In [2]:
import torch
import os
import yaml
import shutil
from ultralytics import YOLO

# -----------------------------
# GPU CHECK (VS Code version)
# -----------------------------
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# -----------------------------
# No Colab, no Drive
# Set your local dataset path here
# Example: r"C:\Users\Jiya\Documents\weapon-dataset"
# -----------------------------
DATASET_PATH = r"C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection"

print("Using dataset path:", DATASET_PATH)
assert os.path.exists(DATASET_PATH), "❌ Dataset path not found! Fix the folder path."


GPU Available: True
GPU Name: NVIDIA GeForce GTX 1650
Using dataset path: C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection


In [3]:
import os

DATASET_ROOT = r"C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection"
# Set your dataset path (modify this to match your Drive structure)
# ----------------------------------
# Local working directory
# This is where model weights, results, runs/ folders will be saved
# ----------------------------------
WORK_DIR = r"C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run"

os.makedirs(WORK_DIR, exist_ok=True)

print(f"Dataset root: {DATASET_ROOT}")
print(f"Working directory: {WORK_DIR}")


Dataset root: C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection
Working directory: C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run


In [4]:
# ============================================================================
# STEP 3: Verify Dataset Structure (VS Code Version)
# ============================================================================

# Paths (these variables were defined in the previous cell)
train_images = os.path.join(DATASET_ROOT, 'train', 'images')
train_labels = os.path.join(DATASET_ROOT, 'train', 'labels')
val_images = os.path.join(DATASET_ROOT, 'val', 'images')
val_labels = os.path.join(DATASET_ROOT, 'val', 'labels')

print("\nVerifying dataset structure:")
print(f"Train images folder exists: {os.path.exists(train_images)}")
print(f"Train labels folder exists: {os.path.exists(train_labels)}")
print(f"Val images folder exists: {os.path.exists(val_images)}")
print(f"Val labels folder exists: {os.path.exists(val_labels)}")

# Count files safely
def count_files(path, exts):
    if not os.path.exists(path):
        return 0
    return len([f for f in os.listdir(path) if f.lower().endswith(exts)])

if os.path.exists(train_images):
    train_img_count = count_files(train_images, ('.jpg', '.png', '.jpeg'))
    train_label_count = count_files(train_labels, ('.txt',))
    print(f"\nTraining images: {train_img_count}")
    print(f"Training labels: {train_label_count}")

if os.path.exists(val_images):
    val_img_count = count_files(val_images, ('.jpg', '.png', '.jpeg'))
    val_label_count = count_files(val_labels, ('.txt',))
    print(f"Validation images: {val_img_count}")
    print(f"Validation labels: {val_label_count}")



Verifying dataset structure:
Train images folder exists: True
Train labels folder exists: True
Val images folder exists: True
Val labels folder exists: True

Training images: 571
Training labels: 571
Validation images: 143
Validation labels: 143


In [5]:
print("\n" + "="*80)
print("CONVERTING DATASET TO SINGLE-CLASS FORMAT (weapon)")
print("="*80)

def convert_to_single_class(label_dir):
    """Convert every label file so all classes become class 0."""
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    converted = 0

    for lf in label_files:
        path = os.path.join(label_dir, lf)

        new_lines = []
        with open(path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    # Always convert class_id → 0
                    x, y, w, h = parts[1:5]
                    new_lines.append(f"0 {x} {y} {w} {h}\n")

        # Rewrite file in-place
        with open(path, 'w') as f:
            f.writelines(new_lines)

        converted += 1

    return converted


train_converted = convert_to_single_class(train_labels)
val_converted = convert_to_single_class(val_labels)

print(f"✓ Converted {train_converted} training labels into class 0")
print(f"✓ Converted {val_converted} validation labels into class 0")
print("\nAll weapons are now labeled as: class 0 → 'weapon'")



CONVERTING DATASET TO SINGLE-CLASS FORMAT (weapon)
✓ Converted 571 training labels into class 0
✓ Converted 143 validation labels into class 0

All weapons are now labeled as: class 0 → 'weapon'


In [6]:
#Create Final YAML (Uses YOLO Built-In Augmentation)
print("\n" + "="*80)
print("CREATING DATASET YAML (Single Class)")
print("="*80)

data_yaml = {
    'path': DATASET_ROOT,
    'train': 'train/images',
    'val': 'val/images',
    'nc': 1,
    'names': ['weapon']
}

yaml_path = os.path.join(WORK_DIR, 'weapon_dataset_single_class.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"\n✓ Dataset YAML created at: {yaml_path}\n")
with open(yaml_path, 'r') as f:
    print(f.read())



CREATING DATASET YAML (Single Class)

✓ Dataset YAML created at: C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\weapon_dataset_single_class.yaml

path: C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection
train: train/images
val: val/images
nc: 1
names:
- weapon



In [7]:
# ============================================================================
# STEP 6: Initialize YOLOv8 Model (Local VS Code)
# ============================================================================

from ultralytics import YOLO
import torch

# Check GPU
print("\nChecking GPU availability...")
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠ No GPU detected. Training will run on CPU.")

# Choose model size
MODEL_SIZE = 'yolov8s.pt'   # small model (good accuracy + speed)

# Load model
model = YOLO(MODEL_SIZE)

print(f"\n✓ Loaded YOLOv8 model: {MODEL_SIZE}")



Checking GPU availability...
CUDA available: True
GPU: NVIDIA GeForce GTX 1650

✓ Loaded YOLOv8 model: yolov8s.pt


In [8]:
# ============================================================================
# STEP 7: Train the Model (4GB GPU Safe Settings)
# ============================================================================

from ultralytics import YOLO
import os

print("\n" + "="*80)
print("STARTING TRAINING (Optimized for 4GB GPU)")
print("="*80)

# Use YOLOv8-nano (very light)
MODEL_SIZE = "yolov8s.pt"
model = YOLO(MODEL_SIZE)

yaml_path = os.path.join(WORK_DIR, "weapon_dataset_single_class.yaml")

results = model.train(
    data=yaml_path,
    epochs=100,            
    imgsz=512,            # lighter than 640
    batch=4,              # small batch for 4GB GPU
    patience=20,
    device=0,
    workers=1,
    project=WORK_DIR,
    name="weapon_detection_single_class",
    exist_ok=True,
    pretrained=True,
    optimizer="AdamW",
    lr0=0.003,
    lrf=0.003,
    weight_decay=0.0005,
    cos_lr=True,
    label_smoothing=0.05,
    
    # LIGHT AUGMENTATIONS (GPU friendly)
    mosaic=0.0,           
    mixup=0.0,
    copy_paste=False,
    fliplr=0.3,
    hsv_h=0.01,
    hsv_s=0.4,
    hsv_v=0.3,
    degrees=10,
    scale=0.4,
)

print("\n✓ Training completed!")
print("="*80)



STARTING TRAINING (Optimized for 4GB GPU)
WARNING 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.232  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=False, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\weapon_dataset_single_class.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.4, hsv_v=0.3, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.003, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yol

In [9]:
# ============================================================================
# STEP 8: Evaluate Model Performance (Local VS Code)
# ============================================================================

from ultralytics import YOLO
import os

print("\nRunning validation on trained model...")

# Load your trained best model
best_model_path = os.path.join(
    WORK_DIR,
    "weapon_detection_single_class",
    "weights",
    "best.pt"
)

# Safety check
assert os.path.exists(best_model_path), "❌ best.pt not found! Training may not have finished."

model = YOLO(best_model_path)

# Run validation
metrics = model.val(
    data=os.path.join(WORK_DIR, "weapon_dataset_single_class.yaml"),
    imgsz=512,
    batch=4,
    device=0
)

# Print metrics safely
print("\nValidation Results:")
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")




Running validation on trained model...
Ultralytics 8.3.232  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)


Traceback (most recent call last):
  File "C:\Users\Jiya\AppData\Local\Programs\Python\Python310\lib\multiprocessing\queues.py", line 247, in _feed
    send_bytes(obj)
  File "C:\Users\Jiya\AppData\Local\Programs\Python\Python310\lib\multiprocessing\connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "C:\Users\Jiya\AppData\Local\Programs\Python\Python310\lib\multiprocessing\connection.py", line 285, in _send_bytes
    ov, err = _winapi.WriteFile(self._handle, buf, overlapped=True)
BrokenPipeError: [WinError 232] The pipe is being closed


Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.40.3 ms, read: 186.8262.6 MB/s, size: 497.0 KB)
val: Scanning C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection\val\labels.cache... 143 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 143/143 35.7Kit/s 0.0ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 36/36 13.2it/s 2.7s0.1s
                   all        143        219      0.919      0.826      0.878      0.604
Speed: 1.1ms preprocess, 11.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to C:\Users\Jiya\runs\detect\val

Validation Results:
mAP50:     0.8775
mAP50-95:  0.6036
Precision: 0.9188
Recall:    0.8263


In [11]:
# ============================================================================
# STEP 8: Test on Sample Images
# ============================================================================

# Get some validation images for testing
val_image_files = [f for f in os.listdir(val_images) if f.endswith(('.jpg', '.png', '.jpeg'))][:5]

print(f"\nTesting on {len(val_image_files)} sample images...")

for img_file in val_image_files:
    img_path = os.path.join(val_images, img_file)

    # Run prediction
    results = model.predict(
        source=img_path,
        conf=0.25,        # Confidence threshold
        iou=0.45,         # NMS IoU threshold
        save=True,        # Save results
        project=WORK_DIR,
        name='predictions',
        exist_ok=True
    )

    print(f"\nProcessed: {img_file}")
    if len(results[0].boxes) > 0:
        print(f"  Detections: {len(results[0].boxes)}")
        for box in results[0].boxes:
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            print(f"    - {names[cls]}: {conf:.2f}")
    else:
        print("  No weapons detected")



Testing on 5 sample images...

image 1/1 /content/drive/MyDrive/Voilence Detection/dataset/weapon_detection/val/images/Automatic Rifle_35.jpeg: 384x640 1 Sniper, 70.0ms
Speed: 2.4ms preprocess, 70.0ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/drive/MyDrive/Voilence Detection/predictions

Processed: Automatic Rifle_35.jpeg
  Detections: 1
    - Sniper: 0.55

image 1/1 /content/drive/MyDrive/Voilence Detection/dataset/weapon_detection/val/images/Automatic Rifle_19.jpeg: 384x640 1 Sword, 25.1ms
Speed: 2.9ms preprocess, 25.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/drive/MyDrive/Voilence Detection/predictions

Processed: Automatic Rifle_19.jpeg
  Detections: 1
    - Sword: 0.81

image 1/1 /content/drive/MyDrive/Voilence Detection/dataset/weapon_detection/val/images/Automatic Rifle_9.jpeg: 448x640 5 Swords, 51.3ms
Speed: 2.8ms preprocess, 51.3ms inference, 1.3ms postprocess per image at sha

In [10]:
# ============================================================================
# STEP 8: Test on Sample Images (Local VS Code)
# ============================================================================

import os
from ultralytics import YOLO

print("\nTesting on sample validation images...")

# Load trained model
best_model_path = os.path.join(
    WORK_DIR,
    "weapon_detection_single_class",
    "weights",
    "best.pt"
)

model = YOLO(best_model_path)

# Single class mapping
names = ["weapon"]

# Pick a few validation images
val_image_files = [
    f for f in os.listdir(val_images)
    if f.lower().endswith(('.jpg', '.png', '.jpeg'))
][:5]

print(f"\nTesting on {len(val_image_files)} images...")

# Create prediction output folder
PRED_DIR = os.path.join(WORK_DIR, "predictions")
os.makedirs(PRED_DIR, exist_ok=True)

for img_file in val_image_files:
    img_path = os.path.join(val_images, img_file)

    # Run prediction
    results = model.predict(
        source=img_path,
        conf=0.25,
        iou=0.45,
        save=True,
        project=WORK_DIR,
        name="predictions",
        exist_ok=True,
        device=0
    )

    print(f"\nProcessed: {img_file}")

    if len(results[0].boxes) > 0:
        print(f"  Detections: {len(results[0].boxes)}")
        for box in results[0].boxes:
            conf = float(box.conf[0])
            print(f"    - weapon: {conf:.2f}")
    else:
        print("  No weapon detected")



Testing on sample validation images...

Testing on 5 images...

image 1/1 C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection\val\images\Automatic Rifle_13.jpeg: 288x512 1 weapon, 35.5ms
Speed: 5.8ms preprocess, 35.5ms inference, 30.9ms postprocess per image at shape (1, 3, 288, 512)
Results saved to C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\predictions

Processed: Automatic Rifle_13.jpeg
  Detections: 1
    - weapon: 0.91

image 1/1 C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\dataset\weapon_detection\val\images\Automatic Rifle_19.jpeg: 320x512 1 weapon, 45.4ms
Speed: 2.2ms preprocess, 45.4ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 512)
Results saved to C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final run\predictions

Processed: Automatic Rifle_19.jpeg
  Detections: 1
    - weapon: 0.92

image 1/1 C:\Users\Jiya\OneDrive - somaiya.edu\Documents\LY Project\final 

In [14]:

# STEP 10: Load and Use Trained Model
# ============================================================================

print("\n" + "=" * 80)
print("USING YOUR TRAINED MODEL")
print("=" * 80)

# Load the best trained model
trained_model = YOLO(best_model_path)

# Comprehensive prediction function
def predict_weapons(image_path, conf_threshold=0.25, save_result=True):
    """
    Detect weapons in an image with detailed output

    Args:
        image_path: Path to image or directory
        conf_threshold: Confidence threshold for detections (0.0-1.0)
        save_result: Whether to save annotated image

    Returns:
        results: Detection results with weapon information
    """
    results = trained_model.predict(
        source=image_path,
        conf=conf_threshold,
        iou=0.45,
        save=save_result,
        project=WORK_DIR,
        name='weapon_predictions',
        exist_ok=True,
        show_labels=True,
        show_conf=True,
        line_width=2
    )

    # Print detailed detection info
    for r in results:
        if len(r.boxes) > 0:
            print(f"\n🎯 Detected {len(r.boxes)} weapon(s) in image:")
            print("-" * 50)

            # Count each weapon type
            from collections import Counter
            weapon_counts = Counter()

            for box in r.boxes:
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                weapon_name = names[cls]
                weapon_counts[weapon_name] += 1

                # Get bounding box coordinates
                x1, y1, x2, y2 = box.xyxy[0].tolist()

                print(f"  • {weapon_name}")
                print(f"    Confidence: {conf:.2%}")
                print(f"    Location: ({int(x1)}, {int(y1)}) to ({int(x2)}, {int(y2)})")
                print()

            print("Summary:")
            for weapon, count in weapon_counts.most_common():
                print(f"  {weapon}: {count}")
        else:
            print(f"\n✓ No weapons detected in image")

    return results

# Test batch prediction on validation set
print("\nTesting batch prediction on validation images...")

# Get some validation images for testing
val_image_files = [f for f in os.listdir(val_images) if f.endswith(('.jpg', '.png', '.jpeg'))][:10]

print(f"Processing {len(val_image_files)} sample images from validation set...\n")

total_detections = 0
images_with_weapons = 0

for i, img_file in enumerate(val_image_files, 1):
    img_path = os.path.join(val_images, img_file)
    print(f"\n[{i}/{len(val_image_files)}] Processing: {img_file}")

    results = trained_model.predict(
        source=img_path,
        conf=0.25,
        iou=0.45,
        save=True,
        project=WORK_DIR,
        name='validation_predictions',
        exist_ok=True
    )

    if len(results[0].boxes) > 0:
        images_with_weapons += 1
        num_weapons = len(results[0].boxes)
        total_detections += num_weapons

        print(f"  ✓ Found {num_weapons} weapon(s):")

        # Group by weapon type
        from collections import Counter
        weapons_found = Counter()

        for box in results[0].boxes:
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            weapon_name = names[cls]
            weapons_found[weapon_name] += 1

        for weapon, count in weapons_found.most_common():
            print(f"    - {weapon}: {count}")
    else:
        print(f"  • No weapons detected")

print("\n" + "=" * 80)
print("BATCH PREDICTION SUMMARY")
print("=" * 80)
print(f"Total images processed: {len(val_image_files)}")
print(f"Images with weapons detected: {images_with_weapons}")
print(f"Images without weapons: {len(val_image_files) - images_with_weapons}")
print(f"Total weapon detections: {total_detections}")
print(f"Average detections per image: {total_detections/len(val_image_files):.2f}")

# Example usage function
print("\n" + "=" * 80)
print("HOW TO USE YOUR MODEL")
print("=" * 80)

example_code = f'''
# Load your trained model
from ultralytics import YOLO
model = YOLO('{best_model_path}')

# Predict on a single image
results = model.predict('path/to/image.jpg', conf=0.25)

# Predict on multiple images
results = model.predict('path/to/folder/', conf=0.25)

# Predict on video
results = model.predict('path/to/video.mp4', conf=0.25)

# Access detection results
for r in results:
    boxes = r.boxes  # Bounding boxes
    for box in boxes:
        cls = int(box.cls[0])  # Class ID
        conf = box.conf[0]     # Confidence
        weapon = {names}[cls]  # Weapon name
        print(f"Detected: {{weapon}} ({{conf:.2%}} confidence)")

# Adjust confidence threshold based on your needs:
# - Higher (0.5-0.7): Fewer false positives, might miss some weapons
# - Lower (0.15-0.25): Catch more weapons, might have false positives
# - Recommended: 0.25-0.4 for security applications
'''

print(example_code)



USING YOUR TRAINED MODEL

Testing batch prediction on validation images...
Processing 10 sample images from validation set...


[1/10] Processing: Automatic Rifle_35.jpeg

image 1/1 /content/drive/MyDrive/Voilence Detection/dataset/weapon_detection/val/images/Automatic Rifle_35.jpeg: 384x640 1 Sniper, 26.6ms
Speed: 3.3ms preprocess, 26.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/drive/MyDrive/Voilence Detection/validation_predictions
  ✓ Found 1 weapon(s):
    - Sniper: 1

[2/10] Processing: Automatic Rifle_19.jpeg

image 1/1 /content/drive/MyDrive/Voilence Detection/dataset/weapon_detection/val/images/Automatic Rifle_19.jpeg: 384x640 1 Sword, 31.9ms
Speed: 3.8ms preprocess, 31.9ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/drive/MyDrive/Voilence Detection/validation_predictions
  ✓ Found 1 weapon(s):
    - Sword: 1

[3/10] Processing: Automatic Rifle_9.jpeg

image 1/1 /content/drive/MyD